In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [2]:
BASE_URL = "https://finviz.com/screener.ashx?v=111&f=cap_largeover,sh_avgvol_o1000&ft=4&r={}"
HEADERS = {"User-Agent": "Mozilla/5.0"}

In [3]:
def extract_table_from_page(page_number):
    url = BASE_URL.format(page_number)
    print(f"🌐 Requesting page {page_number} → {url}")
    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find("table", class_="table-light")

    if not table:
        print(f"❌ Nessuna tabella trovata per pagina {page_number}")
        return []

    rows = table.find_all("tr")[1:]  # skip header
    data = []
    for row in rows:
        cols = row.find_all("td")
        values = [col.text.strip() for col in cols]
        data.append(values)

    print(f"✅ Pagina {page_number} → {len(data)} righe trovate")
    return data


In [4]:
data = extract_table_from_page(1)
pd.DataFrame(data).head()

🌐 Requesting page 1 → https://finviz.com/screener.ashx?v=111&f=cap_largeover,sh_avgvol_o1000&ft=4&r=1
❌ Nessuna tabella trovata per pagina 1


""


In [5]:
pip install finvizfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
from finvizfinance.screener.overview import Overview
import pandas as pd
import os

# Crea la cartella se non esiste
os.makedirs("data", exist_ok=True)

f = Overview()
f.set_filter(filters_dict={
    "Market Cap.": "+Large (over $10bln)",
    "Average Volume": "Over 1M"
})

df = f.screener_view()
df.to_csv("data/fundamentals_finviz.csv", index=False)
print("✅ CSV salvato con successo!")


✅ CSV salvato con successo!#######################-] 34/35 


In [16]:
from finvizfinance.screener.valuation import Valuation

f = Valuation()
f.set_filter(filters_dict={
    "Market Cap.": "+Large (over $10bln)",
    "Average Volume": "Over 1M"
})
df_valuation = f.screener_view()
df_valuation.to_csv("data/fundamentals_valuation.csv", index=False)
print("✅ Valuation salvato!")

✅ Valuation salvato![#############################-] 34/35 


In [17]:
from finvizfinance.screener.financial import Financial

f = Financial()
f.set_filter(filters_dict={
    "Market Cap.": "+Large (over $10bln)",
    "Average Volume": "Over 1M"
})
df_financial = f.screener_view()
df_financial.to_csv("data/fundamentals_financial.csv", index=False)
print("✅ Financial salvato!")

✅ Financial salvato![#############################-] 34/35 


In [18]:
from finvizfinance.screener.technical import Technical

f = Technical()
f.set_filter(filters_dict={
    "Market Cap.": "+Large (over $10bln)",
    "Average Volume": "Over 1M"
})
df_technical = f.screener_view()
df_technical.to_csv("data/fundamentals_technical.csv", index=False)
print("✅ Technical salvato!")

✅ Technical salvato![#############################-] 34/35 


In [21]:
import pandas as pd
import os

# Percorso ai 4 CSV esportati
path = "data/"
file_overview = os.path.join(path, "fundamentals_finviz.csv")
file_valuation = os.path.join(path, "fundamentals_valuation.csv")
file_financial = os.path.join(path, "fundamentals_financial.csv")
file_technical = os.path.join(path, "fundamentals_technical.csv")

# Carica i 4 DataFrame
df_overview = pd.read_csv(file_overview)
df_valuation = pd.read_csv(file_valuation)
df_financial = pd.read_csv(file_financial)
df_technical = pd.read_csv(file_technical)

# 👉 Identifica le colonne comuni (oltre 'Ticker')
common_cols_12 = set(df_overview.columns) & set(df_valuation.columns)
common_cols_13 = set(df_overview.columns) & set(df_financial.columns)
common_cols_14 = set(df_overview.columns) & set(df_technical.columns)

# 👉 Rimuove le colonne duplicate prima del merge, tranne 'Ticker'
cols_to_remove_valuation = list(common_cols_12 - {"Ticker"})
cols_to_remove_financial = list(common_cols_13 - {"Ticker"})
cols_to_remove_technical = list(common_cols_14 - {"Ticker"})

df_valuation = df_valuation.drop(columns=cols_to_remove_valuation)
df_financial = df_financial.drop(columns=cols_to_remove_financial)
df_technical = df_technical.drop(columns=cols_to_remove_technical)

# ✅ Merge progressivo su 'Ticker'
df_combined = df_overview.merge(df_valuation, on="Ticker", how="outer")
df_combined = df_combined.merge(df_financial, on="Ticker", how="outer")
df_combined = df_combined.merge(df_technical, on="Ticker", how="outer")

# 🔎 Opzionale: rimuovi colonne completamente vuote
df_combined.dropna(axis=1, how="all", inplace=True)

# 💾 Salva il dataset finale
output_file = os.path.join(path, "fundamentals_combined.csv")
df_combined.to_csv(output_file, index=False)

print(f"✅ Dataset unificato salvato: {output_file}")
print(f"📊 Dimensioni finali: {df_combined.shape}")


✅ Dataset unificato salvato: data/fundamentals_combined.csv
📊 Dimensioni finali: (685, 43)


In [22]:
import pandas as pd
import numpy as np
import os
import re

# === PARAMETRI CONFIGURABILI ===
input_file = "data/fundamentals_combined.csv"
output_file = "data/fundamentals_cleaned.csv"
soglia_colonne = 30  # % di valori mancanti massima accettata per tenere una colonna
soglia_righe = 50    # % di valori mancanti massima accettata per tenere una riga (opzionale)

# === FUNZIONI UTILI ===

def convert_value(x):
    if isinstance(x, str):
        x = x.strip().replace(",", "")
        if x in ["-", "N/A", ""]:
            return np.nan
        if "%" in x:
            try:
                return float(x.strip("%")) / 100
            except:
                return np.nan
        if "B" in x:
            try:
                return float(x.replace("B", "")) * 1e9
            except:
                return np.nan
        if "M" in x:
            try:
                return float(x.replace("M", "")) * 1e6
            except:
                return np.nan
        if "K" in x:
            try:
                return float(x.replace("K", "")) * 1e3
            except:
                return np.nan
        try:
            return float(x)
        except:
            return np.nan
    return x

# === STEP 1: CARICA IL FILE ===
df = pd.read_csv(input_file)

# === STEP 2: FILTRA COLONNE CON TROPPI NaN ===
missing_percent = df.isnull().mean() * 100
columns_to_keep = missing_percent[missing_percent <= soglia_colonne].index.tolist()
df = df[columns_to_keep]

# === STEP 3: CONVERTI VALORI TESTUALI IN NUMERICI
for col in df.columns:
    if col != "Ticker":
        df[col] = df[col].apply(convert_value)

# === STEP 4: RIMUOVI RIGHE CON TROPPI NaN (opzionale)
row_missing_percent = df.isnull().mean(axis=1) * 100
df = df[row_missing_percent <= soglia_righe]

# === STEP 5: SALVA IL FILE PULITO
os.makedirs("data", exist_ok=True)
df.to_csv(output_file, index=False)

print(f"✅ Dataset pulito salvato in: {output_file}")
print(f"📊 Dimensioni finali: {df.shape}")

✅ Dataset pulito salvato in: data/fundamentals_cleaned.csv
📊 Dimensioni finali: (684, 43)


In [23]:
# === TICKER LIST ===

current_buffett = [  # top holdings attuali
    "AAPL", "KO", "BAC", "AXP", "CVX", "MCO", "OXY", "DVA", "KHC"
]

past_buffett = [  # azioni possedute in passato
    "WFC", "GS", "VZ", "IBM", "JNJ", "PG", "TRV", "WMT"
]

buffett_like = [  # azioni mai in portafoglio ma coerenti (es. brand forte, moat, ROE alto)
    "MSFT", "MA", "ADBE", "COST", "UNH", "V", "PEP", "TXN"
]

# === CLASSIFICAZIONE MULTICLASSE ===

def assign_class(ticker):
    if ticker in current_buffett:
        return 3
    elif ticker in past_buffett:
        return 2
    elif ticker in buffett_like:
        return 1
    else:
        return 0

df["buffett_class"] = df["Ticker"].apply(assign_class)

# === SALVA IL FILE ===
df.to_csv("data/fundamentals_ready_multiclass.csv", index=False)
print("🎯 Dataset multiclass salvato in: data/fundamentals_ready_multiclass.csv")

🎯 Dataset multiclass salvato in: data/fundamentals_ready_multiclass.csv


In [24]:
import pandas as pd

# Carica il file CSV già etichettato
df = pd.read_csv("data/fundamentals_ready_multiclass.csv")

# Rimuovi eventuali colonne identificative e target
exclude_cols = ["Ticker", "buffett_class"]
features = [col for col in df.columns if col not in exclude_cols]

# Mostra le feature
print("🔍 Lista delle feature utilizzabili nel modello:")
for f in features:
    print("-", f)

print(f"\nTotale: {len(features)} feature numeriche")

🔍 Lista delle feature utilizzabili nel modello:
- Company
- Sector
- Industry
- Country
- Market Cap
- P/E
- Price
- Change
- Volume
- Fwd P/E
- PEG
- P/S
- P/B
- P/C
- P/FCF
- EPS this Y
- EPS next Y
- EPS past 5Y
- EPS next 5Y
- Sales past 5Y
- Dividend
- ROA
- ROE
- ROI
- Curr R
- Quick R
- LTDebt/Eq
- Debt/Eq
- Gross M
- Oper M
- Profit M
- Earnings
- Beta
- ATR
- SMA20
- SMA50
- SMA200
- 52W High
- 52W Low
- RSI
- Change from Open
- Gap

Totale: 42 feature numeriche
